
# 00 — Project Overview & Setup (PDM: RUL + CoF)

This notebook wires up the **Predictive Maintenance** project for two tasks:

- **RUL Regression** (CNN, LSTM, XGBoost)
- **Chance of Failure (CoF) Classification** (CNN, LSTM, XGBoost)

It sets project paths, loads configs, verifies data availability, initializes a lightweight experiment registry, and builds a **data dictionary** preview for Line 10/20.


In [1]:

# --- Imports & Path Setup
from pathlib import Path
import sys, os, json
from datetime import datetime
import pandas as pd
import numpy as np
import yaml

# Project root = two levels up from this notebook if you keep the suggested structure
# Fallback: current working directory
NB_PATH = Path.cwd()
ROOT = NB_PATH.parent if NB_PATH.name.lower() in {'rul','cof','notebooks'} else NB_PATH
ROOT = ROOT if (ROOT / 'notebooks').exists() else NB_PATH

DATA_DIR = ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
INTERIM_DIR = DATA_DIR / 'interim'
PROCESSED_DIR = DATA_DIR / 'processed'
CONFIGS_DIR = ROOT / 'configs'
EXPERIMENTS_DIR = ROOT / 'experiments'
SRC_DIR = ROOT / 'src'

for d in [DATA_DIR, RAW_DIR, INTERIM_DIR, PROCESSED_DIR, CONFIGS_DIR, EXPERIMENTS_DIR, SRC_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Allow 'src' imports even if not installed as a package yet
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print(f"ROOT       : {ROOT}")
print(f"DATA_DIR   : {DATA_DIR}")
print(f"CONFIGS_DIR: {CONFIGS_DIR}")
print(f"EXPERIMENTS: {EXPERIMENTS_DIR}")
print(f"SRC        : {SRC_DIR}")


ROOT       : d:\Richard Files\WORK\pdm-project
DATA_DIR   : d:\Richard Files\WORK\pdm-project\data
CONFIGS_DIR: d:\Richard Files\WORK\pdm-project\configs
EXPERIMENTS: d:\Richard Files\WORK\pdm-project\experiments
SRC        : d:\Richard Files\WORK\pdm-project\src


In [2]:

# --- Config helpers & defaults
BASE_CFG = CONFIGS_DIR / 'base.yaml'
TASK_RUL = CONFIGS_DIR / 'task_rul.yaml'
TASK_COF = CONFIGS_DIR / 'task_cof.yaml'
ALGO_CNN = CONFIGS_DIR / 'algo_cnn.yaml'
ALGO_LSTM = CONFIGS_DIR / 'algo_lstm.yaml'
ALGO_XGB = CONFIGS_DIR / 'algo_xgb.yaml'
LINE10 = CONFIGS_DIR / 'line10.yaml'
LINE20 = CONFIGS_DIR / 'line20.yaml'

def ensure_yaml(path: Path, content: dict):
    if not path.exists():
        with path.open('w', encoding='utf-8') as f:
            yaml.safe_dump(content, f, sort_keys=False)
        print(f"Created default config: {path.name}")
    else:
        print(f"Found config: {path.name}")

# Reasonable defaults — adjust as needed later
ensure_yaml(BASE_CFG, {
    'random_seed': 42,
    'time_index': 'timestamp',
    'id_col': 'machine_id',
    'target_rul': 'RUL',
    'target_cof': 'CoF',
    'freq': '1min',  # resample frequency if needed
})

ensure_yaml(TASK_RUL, {
    'task': 'RUL',
    'metrics': ['rmse','mae','r2','nasa','silhouette'],
    'train_val_test_split': {'method': 'time_series_split', 'n_splits': 3}
})

ensure_yaml(TASK_COF, {
    'task': 'CoF',
    'metrics': ['f1','recall','roc_auc'],
    'train_val_test_split': {'method': 'time_series_split', 'n_splits': 3}
})

ensure_yaml(ALGO_CNN, {
    'algo': 'cnn',
    'epochs': 50,
    'batch_size': 128,
    'learning_rate': 1e-3,
})

ensure_yaml(ALGO_LSTM, {
    'algo': 'lstm',
    'epochs': 50,
    'batch_size': 128,
    'learning_rate': 1e-3,
    'hidden_size': 64,
    'num_layers': 2
})

ensure_yaml(ALGO_XGB, {
    'algo': 'xgboost',
    'n_estimators': 500,
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.9,
    'colsample_bytree': 0.9
})

ensure_yaml(LINE10, {'line': 10, 'source': 'csv', 'path': str(RAW_DIR / 'Line10' / 'DM_Machine_Learning_Line_10.csv')})
ensure_yaml(LINE20, {'line': 20, 'source': 'csv', 'path': str(RAW_DIR / 'Line20' / 'DM_Machine_Learning_Line_20.csv')})

def load_yaml(path: Path) -> dict:
    with path.open('r', encoding='utf-8') as f:
        return yaml.safe_load(f)

base_cfg = load_yaml(BASE_CFG)
print("Base config loaded:", base_cfg)


Found config: base.yaml
Found config: task_rul.yaml
Found config: task_cof.yaml
Found config: algo_cnn.yaml
Found config: algo_lstm.yaml
Found config: algo_xgb.yaml
Found config: line10.yaml
Found config: line20.yaml
Base config loaded: {'random_seed': 42, 'time_index': 'timestamp', 'id_col': 'machine_id', 'target_rul': 'RUL', 'target_cof': 'CoF', 'freq': '1min'}


In [3]:

# --- Data discovery & preview for Line 10 and Line 20

from pathlib import Path
import pandas as pd

# Define expected files
mnt_line10 = Path('/mnt/data/DM_Machine_Learning_Line_10.csv')
mnt_line20 = Path('/mnt/data/DM_Machine_Learning_Line_20.csv')

proj_line10 = RAW_DIR / 'Line10' / 'DM_Machine_Learning_Line_10.csv'
proj_line20 = RAW_DIR / 'Line20' / 'DM_Machine_Learning_Line_20.csv'

# Ensure directories exist
proj_line10.parent.mkdir(parents=True, exist_ok=True)
proj_line20.parent.mkdir(parents=True, exist_ok=True)

# Copy from /mnt/data if present (for local or chat attachments)
for mnt_src, proj_dst, line in [
    (mnt_line10, proj_line10, 10),
    (mnt_line20, proj_line20, 20)
]:
    if mnt_src.exists():
        try:
            if not proj_dst.exists():
                proj_dst.write_bytes(mnt_src.read_bytes())
                print(f"✅ Copied Line {line} CSV into project: {proj_dst}")
        except Exception as e:
            print(f"⚠️ Copy skipped for Line {line}, reason:", e)

# Load all available datasets
paths = {
    10: proj_line10,
    20: proj_line20
}

dfs = []
for line, path in paths.items():
    if path.exists():
        try:
            df = pd.read_csv(path)
            df["__line"] = line
            dfs.append(df)
            print(f"✅ Loaded Line {line}: {df.shape}")
        except Exception as e:
            print(f"⚠️ Failed to read Line {line}: {e}")
    else:
        print(f"❌ Missing file for Line {line}: {path}")

# Combine for quick overview
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
    print(f"\n📦 Combined dataset shape: {df_all.shape}")
    display(df_all.head())
else:
    print("⚠️ No Line 10 or Line 20 data found yet.")

# Optional: save quick reference copies
if dfs:
    combined_path = PROCESSED_DIR / 'raw_combined_preview.parquet'
    df_all.to_parquet(combined_path, index=False)
    print(f"💾 Saved combined preview → {combined_path}")


C:\Users\albert.dammara\AppData\Local\Temp\ipykernel_17452\1589781582.py:40: DtypeWarning: Columns (70,71,72,73,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


✅ Loaded Line 10: (349920, 79)


C:\Users\albert.dammara\AppData\Local\Temp\ipykernel_17452\1589781582.py:40: DtypeWarning: Columns (144,145,146,147,149) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


✅ Loaded Line 20: (349920, 153)

📦 Combined dataset shape: (699840, 221)


,Timestamp,Mesin,Energy_Sectors.Extruder_10_Target,EXT_10.Extruder_Load,EXT_10.Machine_Hour,EXT_10.Machine_Run,EXT_10.Melt_Press,EXT_10.Melt_Temp_1,EXT_10.Motor_Extruder_Run,EXT_10.Panel_Humidity,...,EXT_20_Cincinnati.Temp_Setpoint_Dies_16,EXT_20_Cincinnati.Vacuum_Press_Actual,EXT_20_Cincinnati.Vacuum_Press_Setpoint,EXT_20_Cincinnati.Vacuum_Pump_Run,PM_Extruder_20.A_avg,PM_Extruder_20.Frequency,PM_Extruder_20.P,PM_Extruder_20.Power_Factor,PM_Extruder_20.V_avg,PM_Extruder_20.W_tot
0,2025-01-01 00:00:00.000,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-01 00:01:00.000,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-01 00:02:00.000,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-01 00:03:00.000,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-01 00:04:00.000,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


💾 Saved combined preview → d:\Richard Files\WORK\pdm-project\data\processed\raw_combined_preview.parquet


In [4]:

# --- Build data dictionaries for Line 10, Line 20, and Combined (if available)

import numpy as np
import pandas as pd

def make_data_dictionary(df: pd.DataFrame, n_cat_top=10) -> pd.DataFrame:
    info = []
    for col in df.columns:
        s = df[col]
        dtype = str(s.dtype)
        n_missing = int(s.isna().sum())
        missing_pct = float(100 * n_missing / len(s)) if len(s) else 0.0
        nunique = int(s.nunique(dropna=True))

        entry = {
            'column': col,
            'dtype': dtype,
            'n_missing': n_missing,
            'missing_pct': round(missing_pct, 2),
            'n_unique': nunique,
        }

        if np.issubdtype(s.dtype, np.number):
            entry.update({
                'min': s.min(skipna=True),
                'max': s.max(skipna=True),
                'mean': s.mean(skipna=True),
                'std': s.std(skipna=True),
            })
        elif np.issubdtype(s.dtype, np.datetime64):
            entry.update({
                'min': s.min(skipna=True),
                'max': s.max(skipna=True),
            })
        else:
            vc = s.value_counts(dropna=True).head(n_cat_top)
            entry['top_values'] = "; ".join([f"{k}:{int(v)}" for k, v in vc.items()])

        info.append(entry)

    return pd.DataFrame(info)


def _save_and_preview(dd: pd.DataFrame, title: str, path: Path):
    dd.to_csv(path, index=False)
    print(f"Data dictionary saved → {path}")
    try:
        # If available (e.g., in ChatGPT sandbox), this shows a nice table UI.
        from caas_jupyter_tools import display_dataframe_to_user
        display_dataframe_to_user(title, dd)
    except Exception:
        display(dd.head(20))


# --- Per-line + combined
made_any = False

# Line 10
if 'df10' in globals() and df10 is not None and not df10.empty:
    dd10 = make_data_dictionary(df10)
    _save_and_preview(dd10, "Data Dictionary — Line 10", PROCESSED_DIR / 'data_dictionary_line10.csv')
    made_any = True
else:
    print("No Line 10 dataframe available to profile.")

# Line 20 (if you created df20 in the previous cell)
if 'df20' in globals() and df20 is not None and not df20.empty:
    dd20 = make_data_dictionary(df20)
    _save_and_preview(dd20, "Data Dictionary — Line 20", PROCESSED_DIR / 'data_dictionary_line20.csv')
    made_any = True
else:
    print("No Line 20 dataframe available to profile.")

# Combined (df_all is created in the previous “load both lines” cell)
if 'df_all' in globals() and df_all is not None and not df_all.empty:
    dd_all = make_data_dictionary(df_all)
    _save_and_preview(dd_all, "Data Dictionary — Combined (L10+L20)", PROCESSED_DIR / 'data_dictionary_combined.csv')
    made_any = True
else:
    print("No combined dataframe available to profile.")

if not made_any:
    print("Nothing to profile — please load Line 10 and/or Line 20 first.")


No Line 10 dataframe available to profile.
No Line 20 dataframe available to profile.
Data dictionary saved → d:\Richard Files\WORK\pdm-project\data\processed\data_dictionary_combined.csv


,column,dtype,n_missing,missing_pct,n_unique,top_values,min,max,mean,std
0,Timestamp,object,0,0.00,349920,2025-08-31 23:20:00.000:2; 2025-08-31 23:21:00...,NaN,NaN,NaN,NaN
1,Mesin,int64,0,0.00,2,NaN,1.000000e+01,2.000000e+01,1.500000e+01,5.000004e+00
2,Energy_Sectors.Extruder_10_Target,float64,360132,51.46,2,NaN,1.850000e-01,1.930000e-01,1.866387e-01,3.228635e-03
3,EXT_10.Extruder_Load,float64,381427,54.50,6238,NaN,0.000000e+00,9.596000e+01,5.507042e+01,2.330526e+01
4,EXT_10.Machine_Hour,float64,360143,51.46,1,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,EXT_10.Machine_Run,float64,384512,54.94,2,NaN,0.000000e+00,1.000000e+00,8.454054e-01,3.615183e-01
6,EXT_10.Melt_Press,float64,375828,53.70,2672,NaN,0.000000e+00,4.023000e+02,2.617037e+02,1.140031e+02
7,EXT_10.Melt_Temp_1,float64,375825,53.70,2192,NaN,0.000000e+00,3.276700e+03,3.439888e+02,6.982529e+02
8,EXT_10.Motor_Extruder_Run,float64,384512,54.94,2,NaN,0.000000e+00,1.000000e+00,8.454054e-01,3.615183e-01
9,EXT_10.Panel_Humidity,float64,383974,54.87,462,NaN,0.000000e+00,8.130000e+01,5.043909e+01,1.153319e+01


In [5]:

# --- Experiment registry bootstrap
for task in ['RUL', 'CoF']:
    task_dir = EXPERIMENTS_DIR / task
    artifacts = task_dir / 'artifacts'
    task_dir.mkdir(parents=True, exist_ok=True)
    artifacts.mkdir(parents=True, exist_ok=True)
    runs_csv = task_dir / 'runs.csv'
    if not runs_csv.exists():
        cols = [
            'timestamp','task','algo','line','seed','params_json',
            # RUL metrics
            'rmse','mae','r2','nasa','silhouette',
            # CoF metrics
            'f1','recall','roc_auc',
            # bookkeeping
            'dataset_hash','notes','artifact_dir'
        ]
        pd.DataFrame(columns=cols).to_csv(runs_csv, index=False)
        print(f"Created experiment registry: {runs_csv}")
    else:
        print(f"Found experiment registry: {runs_csv}")


Found experiment registry: d:\Richard Files\WORK\pdm-project\experiments\RUL\runs.csv
Found experiment registry: d:\Richard Files\WORK\pdm-project\experiments\CoF\runs.csv


In [6]:

# --- Utility to log experiment runs
def log_run(task:str, algo:str, line:int, seed:int, params:dict,
            metrics:dict, dataset_hash:str='', notes:str='') -> Path:
    task_dir = EXPERIMENTS_DIR / task
    artifacts_dir = task_dir / 'artifacts' / f"{datetime.utcnow().strftime('%Y%m%dT%H%M%S')}_{algo}_L{line}_s{seed}"
    artifacts_dir.mkdir(parents=True, exist_ok=True)

    runs_csv = task_dir / 'runs.csv'
    row = {
        'timestamp': datetime.utcnow().isoformat(),
        'task': task, 'algo': algo, 'line': line, 'seed': seed,
        'params_json': json.dumps(params, ensure_ascii=False),
        'dataset_hash': dataset_hash, 'notes': notes,
        'artifact_dir': str(artifacts_dir)
    }
    # Merge metrics keys that exist in schema
    df_runs = pd.read_csv(runs_csv)
    for k,v in metrics.items():
        if k in df_runs.columns:
            row[k] = v

    df_runs = pd.concat([df_runs, pd.DataFrame([row])], ignore_index=True)
    df_runs.to_csv(runs_csv, index=False)
    print(f"Logged run → {runs_csv.name}: {row['algo']} | {row['task']} | L{line} | seed={seed}")
    return artifacts_dir

print("`log_run` ready. Import this cell into training notebooks (or paste function there).")


`log_run` ready. Import this cell into training notebooks (or paste function there).



## Next steps (linked notebooks)

- `RUL/01_eda_data_prep_RUL.ipynb` — unify time index, handle missingness, align sensors vs. OEE, build/verify **RUL labels**.
- `RUL/02_feature_engineering_RUL.ipynb` — rolling stats, trend/lag, frequency features, normalization; snapshot to `data/processed`.
- `RUL/03_train_eval_RUL.ipynb` (+ `03a/b/c`) — parameterized training for **CNN/LSTM/XGBoost**, log metrics to `experiments/RUL/runs.csv`.
- `RUL/04_model_comparison_RUL.ipynb` — aggregate & visualize (RMSE/MAE/R²/NASA/Silhouette).

We will mirror the flow for **CoF** under `notebooks/CoF/`.
